## Constraint Programming

Constraint programming is a type of declarative programming where you don't code the steps to reach a set of solutions, but you encode the variables to be determined and a set of constraints for success in setting them. In this intro, I'll be using the module python-constraint, which can be installed with
```python -m pip install python-contraint```
or
```pip install python-constraint```

As an example, I'll use an arbitrary, simple, and not very useful problem where we need to determine x and y where x is 1, 2, or 3, and y is an integer between 0 and 9 inclusive, and the sum of the 2 is greater than or equal to 5.

In [1]:
import constraint

We will define the variables x and y, and their acceptable values, and the additional constraint that their sum be greater than or equal to 5.

In [2]:
problem = constraint.Problem()

problem.addVariable('x', [1,2,3])
problem.addVariable('y', range(10))

def our_constraint(x, y):
    if x + y >= 5:
        return True

problem.addConstraint(our_constraint, ['x', 'y'])

solutions = problem.getSolutions()
solutions

[{'x': 3, 'y': 9},
 {'x': 3, 'y': 8},
 {'x': 3, 'y': 7},
 {'x': 3, 'y': 6},
 {'x': 3, 'y': 5},
 {'x': 3, 'y': 4},
 {'x': 3, 'y': 3},
 {'x': 3, 'y': 2},
 {'x': 2, 'y': 9},
 {'x': 2, 'y': 8},
 {'x': 2, 'y': 7},
 {'x': 2, 'y': 6},
 {'x': 2, 'y': 5},
 {'x': 2, 'y': 4},
 {'x': 2, 'y': 3},
 {'x': 1, 'y': 9},
 {'x': 1, 'y': 8},
 {'x': 1, 'y': 7},
 {'x': 1, 'y': 6},
 {'x': 1, 'y': 5},
 {'x': 1, 'y': 4}]

You'll notice that the result is a list of dictionaries, each one giving the results for each variable in that solution.

The values don't have to be integers or even numbers, they just have to be testable against the constraints that you have. They can be lists, or dictionaries, or whatever. I'm mostly using integers because it's easy to demonstrate with them.

## Digit Letter Puzzles

There are puzzles where you need to solve a problem like
```
 TWO
+TWO
-----
FOUR
```
Where each unique letter stands for a different digit, and when the digits are substituted, the addition problem as noted gives a correct answer. Initial digits (the T and the F) cannot be 0, and no two letters can have the same digit value.
We set a variable for each letter, with the valid values, and add the constraint that the addition problem is correct.
A constraint is an executable that takes variables as parameters, and returns True if the constraint is met.
It's easy to use strings for multiple variable names, since they are iterable.

In [3]:
problem = constraint.Problem()

problem.addVariables("TF", range(1,10))
problem.addVariables("WOUR", range(10))

def sum_constraint(t, w, o, f, u, r):
    if 2*(t*100 + w*10 + o) == f*1000 + o*100 +u*10 + r:
        return True
    
problem.addConstraint(sum_constraint, "TWOFUR")

problem.addConstraint(constraint.AllDifferentConstraint())

solutions = problem.getSolutions()
solutions

[{'F': 1, 'O': 5, 'R': 0, 'T': 7, 'U': 3, 'W': 6},
 {'F': 1, 'O': 4, 'R': 8, 'T': 7, 'U': 6, 'W': 3},
 {'F': 1, 'O': 7, 'R': 4, 'T': 8, 'U': 3, 'W': 6},
 {'F': 1, 'O': 6, 'R': 2, 'T': 8, 'U': 9, 'W': 4},
 {'F': 1, 'O': 6, 'R': 2, 'T': 8, 'U': 7, 'W': 3},
 {'F': 1, 'O': 8, 'R': 6, 'T': 9, 'U': 5, 'W': 2},
 {'F': 1, 'O': 8, 'R': 6, 'T': 9, 'U': 7, 'W': 3}]

Those answers aren't easy to understand, so let's format it up to make it easier. We create a string with the replacements for TWO and FOUR

In [4]:
for s in solutions:
    d = {}
    for key,val in s.items():
        d[key] = str(val)
    two = "".join([d[x] for x in 'TWO'])
    four = "".join([d[x] for x in 'FOUR'])
    print(" {0}\n {0}\n----\n{1}\n".format(two, four))

 765
 765
----
1530

 734
 734
----
1468

 867
 867
----
1734

 846
 846
----
1692

 836
 836
----
1672

 928
 928
----
1856

 938
 938
----
1876



### 4 x 4 magic square
Here's a bit more ambitious problem with a set of constraints. Form a standard minimal 4x4 magic square.

```
a b c d
e f g h
i j k l
m n o p
```
All rows(abcd, efgh, ...), columns(aeim, ...), and diagonals(afkp, dgjm) add up to the same number. Each entry is a number between 1 and 16, and no number is repeated.
That sum has to be 34 == sum(abcdefghijklmnop)/4; (16+1)(16)/(2*4)

In [6]:
SUM = 34
from constraint import (Problem, AllDifferentConstraint, 
                        ExactSumConstraint)

problem = Problem()

# When I ran the full problem, it took far too long to get the answer
# to use in a demonstration, so I've set the first few values in the
# square to values that produce some of the possible magic squares.
problem.addVariable(0, [1])
problem.addVariable(1, [15])
problem.addVariable(2, [14])
#problem.addVariable(3, [4])
#problem.addVariable(4, [10])
#problem.addVariable(5, [11])
#problem.addVariable(6, [8])
#problem.addVariable(7, [5])
problem.addVariables(range(3, 16), range(1, 16+1)) # variables are ints

problem.addConstraint(AllDifferentConstraint(), range(0,16))
# Diagonals
problem.addConstraint(ExactSumConstraint(SUM), [0,5,10,15])
problem.addConstraint(ExactSumConstraint(SUM), [3,6,9,12])

for row in range(4):
    problem.addConstraint(
        ExactSumConstraint(SUM), 
        [row*4+i for i in range(4)])
    
for col in range(4):
    problem.addConstraint(
        ExactSumConstraint(SUM),
        [col+4*i for i in range(4)])
    

solutions = problem.getSolutions()
print("There {} {} solution(s)".format("is" if len(solutions)==1 else "are", len(solutions)))
for i,s in enumerate(solutions, start=1):
    print("\nSolution {}".format(i))
    for row in range(4):
        print([s[row*4+i] for i in range(4)])

There are 10 solution(s)

Solution 1
[1, 15, 14, 4]
[12, 6, 7, 9]
[8, 10, 11, 5]
[13, 3, 2, 16]

Solution 2
[1, 15, 14, 4]
[11, 8, 5, 10]
[6, 9, 12, 7]
[16, 2, 3, 13]

Solution 3
[1, 15, 14, 4]
[10, 8, 5, 11]
[7, 9, 12, 6]
[16, 2, 3, 13]

Solution 4
[1, 15, 14, 4]
[7, 9, 6, 12]
[10, 8, 11, 5]
[16, 2, 3, 13]

Solution 5
[1, 15, 14, 4]
[12, 9, 6, 7]
[5, 8, 11, 10]
[16, 2, 3, 13]

Solution 6
[1, 15, 14, 4]
[8, 10, 11, 5]
[12, 6, 7, 9]
[13, 3, 2, 16]

Solution 7
[1, 15, 14, 4]
[5, 11, 8, 10]
[12, 6, 9, 7]
[16, 2, 3, 13]

Solution 8
[1, 15, 14, 4]
[10, 11, 8, 5]
[7, 6, 9, 12]
[16, 2, 3, 13]

Solution 9
[1, 15, 14, 4]
[7, 12, 9, 6]
[10, 5, 8, 11]
[16, 2, 3, 13]

Solution 10
[1, 15, 14, 4]
[6, 12, 9, 7]
[11, 5, 8, 10]
[16, 2, 3, 13]


## Other applications

These were all pretty artificial or game like examples, but there are real world examples like:

 - Locating devices in different locations
 - Assigning resources to projects
 - Packing things in containers
 - Finding paths through a network and allocating traffic to them

### Built in constraints
- AllDifferentConstraint
- AllEqualConstraint
- MaxSumConstraint
- ExactSumConstraint
- MinSumConstraint
- InSetConstraint
- NotinSetConstraint
- SomeInSetConstraint
- SomeNotInSetConstraint

There's lots more to learn about the package. It's possible to use different solvers: Backtracking, Recursive backtracking, and Minimum conflicts.